In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import *
from tqdm import tqdm
from numba import njit

def add_periodic_boundaries(arr):
    # takes an (n-2, m-2) array and returns (n, m) array
    rows, cols = arr.shape
    output = np.zeros((rows+2, cols+2), dtype=arr.dtype)

    # Fill the output array with the periodic boundaries
    output[1:-1, 1:-1] = arr # center
    output[0:1, 1:-1] = arr[-1:,:] # first row
    output[-1:, 1:-1] = arr[0:1,:] # last row
    output[1:-1, 0:1] = arr[:,-1:] # first column
    output[1:-1, -1:] = arr[:,0:1] # last column

    return output

plt.rcParams["figure.figsize"] = [14, 8]
plt.rcParams["figure.dpi"] = 100
plt.set_cmap("binary_r")

<Figure size 1400x800 with 0 Axes>